# Fit UMAP To Data + Save Manifold at Low Dimension

In [ ]:
import torch
import numpy as np
from pathlib import Path
import umap.umap_ as umap

import joblib  # for saving the UMAP model

# --- CONFIG ---
reduced_dir = Path("../Results/EmbeddingDataReduced")
umap_dir = Path("../Results/EmbeddingDataUMAP")
umap_dir.mkdir(parents=True, exist_ok=True)

# UMAP parameters (tunable by user)
n_neighbors = 11
min_dist = 0.1
n_components = 10   # change to 3 if you want 3D embeddings
metric = "euclidean"
random_state = 42

# --- COLLECT REDUCED BATCH FILES ---
batch_files = sorted(reduced_dir.glob("batch_*.pt"))
print(f"Found {len(batch_files)} reduced batch files")

# --- INIT UMAP ---
umap_model = umap.UMAP(
    n_neighbors=n_neighbors,
    min_dist=min_dist,
    n_components=n_components,
    metric=metric,
    random_state=random_state,
    verbose=True,
)

# --- FIT UMAP ON ALL DATA ---
print("Fitting UMAP model...")
# Load all batches into memory for fit (UMAP does not support partial_fit)
all_data = []
for f in batch_files:
    batch_tensor = torch.load(f, weights_only=True)
    all_data.append(batch_tensor.numpy())
all_data = np.vstack(all_data)
print(f"Total data for UMAP fit: {all_data.shape}")

umap_model.fit(all_data)
print("UMAP model trained.")

# --- SAVE UMAP MODEL ---
joblib.dump(umap_model, umap_dir / "umap_model.pkl")
print("Saved UMAP model.")

# --- TRANSFORM & SAVE EACH BATCH ---
for f in batch_files:
    print(f"Transforming {f.name} ...")
    batch_tensor = torch.load(f, weights_only=True)
    reduced = umap_model.transform(batch_tensor.numpy())
    reduced_tensor = torch.from_numpy(reduced).to(torch.float32)

    out_file = umap_dir / f.name
    torch.save(reduced_tensor, out_file)
    print(f"Saved UMAP batch to {out_file}")


### Test UMAP data was saved properly

In [ ]:
import torch
from pathlib import Path

# --- CONFIG ---
reduced_dir = Path("../Results/EmbeddingDataUMAP")

# --- CHECK ---
reduced_files = sorted(reduced_dir.glob("batch_*.pt"))
if reduced_files:
    first_file = reduced_files[0]
    print(f"Loading {first_file.name} ...")
    reduced_tensor = torch.load(first_file, weights_only=True)

    print(f"Type: {type(reduced_tensor)}")
    print(f"Shape: {reduced_tensor.shape}")
    print(f"Dtype: {reduced_tensor.dtype}")
    print(f"First 3 rows:\n{reduced_tensor[:3]}")
    print(f"Min/max values: {reduced_tensor.min().item():.6f} / {reduced_tensor.max().item():.6f}")
else:
    print("No reduced batch files found in the directory.")

# Run HDBSCAN To Detect Clusters

In [ ]:
import torch
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # registers 3D projection
import hdbscan
from collections import Counter

# --- CONFIG ---
umap_dir = Path("../Results/EmbeddingDataUMAP")

# --- LOAD ALL BATCHES ---
batch_files = sorted(umap_dir.glob("batch_*.pt"))
all_embeds = []

for f in batch_files:
    print(f"Loading {f.name} ...")
    batch_tensor = torch.load(f, weights_only=True)
    all_embeds.append(batch_tensor.numpy())

all_embeds = np.vstack(all_embeds)  # shape: (total_points, n_dim)
print("Final shape:", all_embeds.shape)

# --- RUN HDBSCAN ---
clusterer = hdbscan.HDBSCAN(
    min_cluster_size=30,  # smaller clusters allowed
    min_samples=10        # fewer points needed to avoid noise
)
cluster_labels = clusterer.fit_predict(all_embeds)
counts = Counter(cluster_labels)


# --- SUMMARY STATISTICS ---
total_points = len(cluster_labels)
noise_points = counts.get(-1, 0)

# Exclude noise for cluster size stats
cluster_sizes = [count for cid, count in counts.items() if cid != -1]
num_clusters = len(cluster_sizes)
mean_size = np.mean(cluster_sizes) if cluster_sizes else 0
std_size = np.std(cluster_sizes) if cluster_sizes else 0
noise_pct = (noise_points / total_points) * 100


print(f"Total points: {total_points}")
print(f"Noise points: {noise_points}")
print(f"Noise percentage: {noise_pct:.2f}%")
print(f"Clusters found (excluding noise): {num_clusters}")
print(f"Mean cluster size: {mean_size:.2f}")
print(f"Std cluster size: {std_size:.2f}")
print(f"Max cluster size: {max(cluster_sizes)}")
print(f"Min cluster size: {min(cluster_sizes)}")



In [ ]:
# --- Data Prep ---
cluster_sizes = [count for cid, count in counts.items() if cid != -1]
cluster_ids = [cid for cid in counts.keys() if cid != -1]

# ------------------------------------------------------------------
# 1. Histogram of cluster sizes
# ------------------------------------------------------------------
plt.figure(figsize=(10, 6))
plt.hist(cluster_sizes, bins=30, edgecolor="black")
plt.xlabel("Cluster size")
plt.ylabel("Number of clusters")
plt.title("Histogram of Cluster Sizes")
plt.show()


In [ ]:
# ------------------------------------------------------------------
# 2. PDF (density estimate) of cluster sizes
# ------------------------------------------------------------------
plt.figure(figsize=(10, 6))
counts_hist, bins = np.histogram(cluster_sizes, bins=30, density=True)
bin_centers = 0.5 * (bins[1:] + bins[:-1])
plt.plot(bin_centers, counts_hist, drawstyle="steps-mid")
plt.xlabel("Cluster size")
plt.ylabel("Probability density")
plt.title("Approximate PDF of Cluster Sizes")
plt.grid(True)
plt.show()

In [ ]:

# ------------------------------------------------------------------
# 3. Randomized 2D Bubble Chart (biggest clusters first)
# ------------------------------------------------------------------
plt.figure(figsize=(10, 8))

# Sort clusters by size (largest first)
sorted_sizes = sorted(cluster_sizes, reverse=True)

# Random positions for each cluster
rng = np.random.default_rng(seed=42)  # reproducible random layout
x = rng.uniform(0, 100, len(sorted_sizes))
y = rng.uniform(0, 100, len(sorted_sizes))

# Bubble radii scaled by cluster size
scale_factor = 0.5  # adjust this for bigger/smaller bubbles
bubble_sizes = np.array(sorted_sizes) * scale_factor

plt.scatter(x, y, s=bubble_sizes, alpha=0.5, c="tab:blue", edgecolors="black")

plt.xlabel("Random X")
plt.ylabel("Random Y")
plt.title("Random Bubble Plot of Cluster Sizes (largest clusters drawn bigger)")
plt.axis("equal")
plt.show()

# T-SNE to 3d for visualization

In [ ]:
import torch
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401, registers 3D projection
from sklearn.manifold import TSNE
import hdbscan
from collections import Counter



# --- RUN TSNE TO 3D ---
print("Running t-SNE to 3D...")
tsne = TSNE(
    n_components=3,
    random_state=42,
    perplexity=30,
    init="pca",
    learning_rate="auto"
)
embeds_3d = tsne.fit_transform(all_embeds)
print("t-SNE shape:", embeds_3d.shape)


In [ ]:
# --- 3D PLOT WITH CLUSTER COLORS ---
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection="3d")

# Use cluster labels as colors (noise = gray)
colors = np.where(cluster_labels == -1, -999, cluster_labels)

scatter = ax.scatter(
    embeds_3d[:, 0],
    embeds_3d[:, 1],
    embeds_3d[:, 2],
    c=colors,
    cmap="tab20",
    s=2,
    alpha=0.7
)

ax.set_xlabel("t-SNE-1")
ax.set_ylabel("t-SNE-2")
ax.set_zlabel("t-SNE-3")
ax.set_title("t-SNE Projection Colored by HDBSCAN Clusters")
plt.show()


In [ ]:
# --- FILTER NON-NOISE POINTS ---
mask = cluster_labels != -1
embeds_3d_non_noise = embeds_3d[mask]
labels_non_noise = cluster_labels[mask]

# --- 3D PLOT WITH CLUSTER COLORS (NON-NOISE ONLY) ---
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection="3d")

scatter = ax.scatter(
    embeds_3d_non_noise[:, 0],
    embeds_3d_non_noise[:, 1],
    embeds_3d_non_noise[:, 2],
    c=labels_non_noise,
    cmap="tab20",
    s=2,
    alpha=0.7
)

ax.set_xlabel("t-SNE-1")
ax.set_ylabel("t-SNE-2")
ax.set_zlabel("t-SNE-3")
ax.set_title("t-SNE Projection (Non-Noise HDBSCAN Clusters)")
plt.show()


In [ ]:
# --- CHOOSE TOP-K CLUSTERS ---
k = 40  # change this as needed

# Count cluster sizes
unique_labels, counts = np.unique(cluster_labels[cluster_labels != -1], return_counts=True)
sorted_clusters = [lab for lab, _ in sorted(zip(unique_labels, counts), key=lambda x: x[1], reverse=True)]

# Keep only top-k
top_k_clusters = set(sorted_clusters[:k])

mask = np.isin(cluster_labels, list(top_k_clusters))
embeds_3d_topk = embeds_3d[mask]
labels_topk = cluster_labels[mask]

# --- 3D PLOT WITH CLUSTER COLORS (TOP-K ONLY) ---
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection="3d")

scatter = ax.scatter(
    embeds_3d_topk[:, 0],
    embeds_3d_topk[:, 1],
    embeds_3d_topk[:, 2],
    c=labels_topk,
    cmap="tab20",
    s=2,
    alpha=0.7
)

ax.set_xlabel("t-SNE-1")
ax.set_ylabel("t-SNE-2")
ax.set_zlabel("t-SNE-3")
ax.set_title(f"t-SNE Projection (Top {k} Largest HDBSCAN Clusters)")
plt.show()


# HDBSCAN Centroids -> Distance Clusters (with no noise)

In [ ]:
# --- META-CLUSTERING WITH AGGLOMERATIVE ---
import numpy as np
from sklearn.cluster import AgglomerativeClustering
from scipy.spatial import distance_matrix

# --- USER HYPERPARAMETERS ---
# You can toggle these for experiments:
USE_CENTROIDS = True         # if False, use medoids (slower, more robust)
LINKAGE = "ward"             # "ward", "complete", "average", "single"
N_META = None                # if None, use DIST_THRESHOLD
DIST_THRESHOLD = 5.0         # cut dendrogram at this distance if N_META is None

# --- COLLECT NON-NOISE CLUSTERS ---
hdb_labels = cluster_labels  # from your HDBSCAN run
unique_labels = sorted([lbl for lbl in np.unique(hdb_labels) if lbl != -1])

# --- COMPUTE CENTROIDS OR MEDOIDS ---
summaries = []
label_to_index = {}
for i, lbl in enumerate(unique_labels):
    members = all_embeds[hdb_labels == lbl]
    if USE_CENTROIDS:
        summary = members.mean(axis=0)
    else:
        # medoid: point with minimum total distance to others
        dists = distance_matrix(members, members)
        medoid_idx = np.argmin(dists.sum(axis=1))
        summary = members[medoid_idx]
    summaries.append(summary)
    label_to_index[lbl] = i
summaries = np.vstack(summaries)

# --- RUN AGGLOMERATIVE ---
agg = AgglomerativeClustering(
    n_clusters=N_META,
    distance_threshold=None if N_META else DIST_THRESHOLD,
    linkage=LINKAGE
)
meta_labels = agg.fit_predict(summaries)

# --- MAP HDBSCAN CLUSTERS TO META-CLUSTERS ---
cluster_to_meta = {lbl: int(meta_labels[idx]) for lbl, idx in label_to_index.items()}

# --- ASSIGN EVERY POINT TO A META-CLUSTER ---
final_labels = np.full_like(hdb_labels, fill_value=-1)
for lbl in unique_labels:
    final_labels[hdb_labels == lbl] = cluster_to_meta[lbl]

# --- HANDLE NOISE POINTS (-1) ---
noise_mask = (hdb_labels == -1)
if noise_mask.any():
    noise_points = all_embeds[noise_mask]
    dists = distance_matrix(noise_points, summaries)
    nearest_idx = np.argmin(dists, axis=1)
    final_labels[noise_mask] = meta_labels[nearest_idx]

print("Final meta-cluster labels shape:", final_labels.shape)
print("Unique meta-clusters:", np.unique(final_labels))




In [ ]:
# --- DENDROGRAM VISUALIZATION ---
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import linkage, dendrogram

# Build hierarchical linkage on the summaries (centroids/medoids)
Z = linkage(summaries, method=LINKAGE)

plt.figure(figsize=(12, 6))
dendro = dendrogram(
    Z,
    labels=unique_labels,     # so each leaf corresponds to original HDBSCAN cluster
    leaf_rotation=90,
    leaf_font_size=10,
    color_threshold=0,        # let us control colors later if needed
)

# Draw horizontal cut line (same as DIST_THRESHOLD from your hyperparams)
if N_META is None:
    plt.axhline(y=DIST_THRESHOLD, c="red", ls="--", lw=2)

plt.title("Agglomerative Clustering Dendrogram of HDBSCAN Summaries")
plt.xlabel("Original HDBSCAN Clusters")
plt.ylabel("Distance")
plt.tight_layout()
plt.show()



In [ ]:
# --- REPRODUCTION / HOLDOUT TESTS FOR AGGLOMERATIVE -> INFERENCE ---
import numpy as np
from collections import Counter, defaultdict
from scipy.spatial import distance_matrix
from sklearn.cluster import AgglomerativeClustering

# Helper: assign new points to meta-clusters using nearest summary
def assign_new_points(new_points, summaries, meta_labels):
    dists = distance_matrix(new_points, summaries)  # (n_new, n_summaries)
    nearest_idx = np.argmin(dists, axis=1)
    return meta_labels[nearest_idx], nearest_idx  # returns meta ids and which summary index chosen

# ------------------------------
# 1) POINT-LEVEL RECONSTRUCTION
# ------------------------------
def point_reconstruction_test(all_embeds, final_labels, summaries, meta_labels,
                              sample_size=1000, random_state=0):
    """
    Randomly sample points and test if nearest-summary assignment reproduces final_labels.
    """
    rng = np.random.RandomState(random_state)
    n = all_embeds.shape[0]
    sample_size = min(sample_size, n)
    idx = rng.choice(n, size=sample_size, replace=False)
    sampled_points = all_embeds[idx]
    pred_meta, nearest_summary_idx = assign_new_points(sampled_points, summaries, meta_labels)
    true_meta = final_labels[idx]

    acc = (pred_meta == true_meta).mean()
    mismatches = np.where(pred_meta != true_meta)[0]

    print(f"[Point test] Sampled {sample_size} points. Accuracy = {acc:.4f}. Mismatches = {len(mismatches)}")
    if len(mismatches) > 0:
        # show up to 10 mismatch examples with helpful context
        print("Examples (index_in_sample, global_index, true_meta, pred_meta, nearest_summary_idx):")
        for ii in mismatches[:10]:
            gi = idx[ii]
            print(ii, gi, int(true_meta[ii]), int(pred_meta[ii]), int(nearest_summary_idx[ii]))
    return {"accuracy": acc, "n_sample": sample_size, "mismatch_indices_sample": mismatches, "sample_global_idx": idx}

# ------------------------------
# 2) SUMMARY / CLUSTER HOLDOUT
# ------------------------------
def summary_holdout_test(unique_labels, summaries, meta_labels,
                         holdout_k=5, random_state=0):
    """
    Hold out some HDBSCAN clusters (their summaries). Re-fit agglomerative on remaining summaries,
    map new meta clusters back to original meta labels (majority vote), then evaluate:
      - how well held-out summaries map back to original meta labels
    """
    rng = np.random.RandomState(random_state)
    n_summ = summaries.shape[0]
    holdout_k = min(holdout_k, n_summ - 1)
    # choose holdout by summary index (these correspond to unique_labels order)
    holdout_indices = rng.choice(n_summ, size=holdout_k, replace=False)
    remain_indices = np.array([i for i in range(n_summ) if i not in set(holdout_indices)], dtype=int)

    summaries_remain = summaries[remain_indices]
    orig_meta_remain = meta_labels[remain_indices]  # original meta labels for these summaries

    # Re-fit agglomerative on remaining summaries using the same hyperparams (LINKAGE / DIST_THRESHOLD / N_META)
    agg_retrain = AgglomerativeClustering(
        n_clusters=N_META,
        distance_threshold=None if N_META else DIST_THRESHOLD,
        linkage=LINKAGE
    )
    new_meta_remain = agg_retrain.fit_predict(summaries_remain)  # labels for each remaining summary

    # Build mapping: new_meta_id -> original meta_id (majority vote among members)
    new_to_orig = {}
    for nm in np.unique(new_meta_remain):
        member_idxs = np.where(new_meta_remain == nm)[0]
        orig_labels_for_members = orig_meta_remain[member_idxs]
        # majority vote
        most_common_orig = Counter(orig_labels_for_members).most_common(1)[0][0]
        new_to_orig[int(nm)] = int(most_common_orig)

    # Assign each held-out summary to nearest remaining summary -> new_meta -> mapped back to original meta
    held_summaries = summaries[holdout_indices]
    d = distance_matrix(held_summaries, summaries_remain)  # (n_holdout, n_remain)
    nearest_remain_idx = np.argmin(d, axis=1)             # indices into summaries_remain
    assigned_new_meta = new_meta_remain[nearest_remain_idx]
    assigned_orig_meta_via_map = np.array([new_to_orig[int(x)] for x in assigned_new_meta])

    # Ground truth for held-out summaries:
    true_orig_meta_holdout = meta_labels[holdout_indices]

    # Compute accuracy
    acc_holdout_summ = (assigned_orig_meta_via_map == true_orig_meta_holdout).mean()

    print(f"[Summary holdout] Held out {len(holdout_indices)} summaries. Accuracy (summary-level) = {acc_holdout_summ:.4f}")

    # Return detailed info so user can inspect mismatches
    results = {
        "holdout_summary_indices": holdout_indices,
        "nearest_remain_idx": nearest_remain_idx,
        "assigned_new_meta": assigned_new_meta,
        "assigned_orig_meta_via_map": assigned_orig_meta_via_map,
        "true_orig_meta_holdout": true_orig_meta_holdout,
        "accuracy_summary_level": acc_holdout_summ,
        "new_to_orig_mapping": new_to_orig,
        "remain_indices": remain_indices,
        "new_meta_remain": new_meta_remain
    }
    return results

# ------------------------------
# USAGE EXAMPLES (run these lines)
# ------------------------------
# 1) Point-level test (fast)
pt_test_res = point_reconstruction_test(
    all_embeds=all_embeds,
    final_labels=final_labels,
    summaries=summaries,
    meta_labels=meta_labels,
    sample_size=1000,
    random_state=42
)

# 2) Summary-holdout test (re-fits agglomerative on remaining summaries)
holdout_res = summary_holdout_test(
    unique_labels=unique_labels,
    summaries=summaries,
    meta_labels=meta_labels,
    holdout_k=5,
    random_state=42
)

# Optional deeper check: for each held-out cluster, evaluate point-level assignment accuracy
def heldout_points_eval(all_embeds, hdb_labels, holdout_summary_indices, unique_labels,
                        summaries_remain, new_meta_remain, new_to_orig_map):
    """
    For each held-out HDBSCAN cluster index (index into summaries / unique_labels),
    evaluate the point-level assignment accuracy when assigning points to remaining summaries.
    """
    per_cluster_stats = {}
    for sidx in holdout_summary_indices:
        original_hdb_label = unique_labels[sidx]
        members_mask = (hdb_labels == original_hdb_label)
        pts = all_embeds[members_mask]
        if pts.shape[0] == 0:
            per_cluster_stats[original_hdb_label] = {"n_points": 0}
            continue
        d = distance_matrix(pts, summaries_remain)
        nearest_idx = np.argmin(d, axis=1)
        assigned_new_meta = new_meta_remain[nearest_idx]
        assigned_orig_meta = np.array([new_to_orig_map[int(x)] for x in assigned_new_meta])
        true_meta = final_labels[members_mask]
        acc = (assigned_orig_meta == true_meta).mean()
        per_cluster_stats[original_hdb_label] = {
            "n_points": pts.shape[0],
            "accuracy": float(acc),
            "examples": {
                "true_meta_sample": true_meta[:5].tolist(),
                "assigned_meta_sample": assigned_orig_meta[:5].tolist()
            }
        }
    return per_cluster_stats

# If you want to run the point-level heldout evaluation, prepare args from holdout_res:
# build summaries_remain and new_meta_remain using holdout_res info:
remain_indices = holdout_res["remain_indices"]
summaries_remain = summaries[remain_indices]
new_meta_remain = holdout_res["new_meta_remain"]
new_to_orig_map = holdout_res["new_to_orig_mapping"]
per_cluster_stats = heldout_points_eval(all_embeds, hdb_labels,
                                        holdout_res["holdout_summary_indices"],
                                        unique_labels,
                                        summaries_remain, new_meta_remain, new_to_orig_map)

# Print brief summary
print("\nPer-heldout-cluster point-level stats (up to 10 clusters):")
for i, (cl, st) in enumerate(per_cluster_stats.items()):
    if i >= 10: break
    print(cl, st)
